In [ ]:
%load_ext dotenv
%dotenv

Perform the following steps to set up an Azure Computer Vision service. In this chapter, we will use the [Azure CLI a lot to create and delete our resources](https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account-cli?tabs=linux). Cognitive Service you setup, then you might be asked to provision the service using the Azure Portal as you need to accept the Responsible AI terms. You will likely see this error message:
```
This subscription cannot create ComputerVision until you agree to Responsible AI terms for this resource. You can agree to Responsible AI terms by creating a resource through the Azure Portal then trying again. For more detail go to https://go.microsoft.com/fwlink/?linkid=2164911
```
First, perform a login according to step 1. As you may have more than one subscriptions in your directory, we set the standard subscription that Azure CLI will use in 2. Finally, there are individual setup and destroy scripts for each service.

1. `az login`
2. `az account set --subscription "<YOUR SUBSCRIPTION ID>"`
3. `./ocr-setup.sh`

# OCR Text recognition

## Links to documentation

* https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/quickstarts-sdk/client-library?tabs=visual-studio&pivots=programming-language-python
* https://docs.microsoft.com/en-us/python/api/overview/azure/cognitiveservices-vision-computervision-readme?view=azure-python

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials

import os
import time

region = os.environ["ACCOUNT_REGION"]
key = os.environ["ACCOUNT_KEY"]

credentials = CognitiveServicesCredentials(key)
client = ComputerVisionClient(
    endpoint="https://" + region + ".api.cognitive.microsoft.com/",
    credentials=credentials
)

# import models
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes

url = "https://github.com/Azure-Samples/cognitive-services-python-sdk-samples/raw/master/samples/vision/images/make_things_happen.jpg"
raw = True
numberOfCharsInOperationId = 36

# SDK call
rawHttpResponse = client.read(url, language="en", raw=True)

# Get ID from returned headers
operation_location = rawHttpResponse.headers["Operation-Location"]
id_location = len(operation_location) - numberOfCharsInOperationId
operation_id = operation_location[id_location:]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    result = client.get_read_result(operation_id)
    if result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Get data
if result.status == OperationStatusCodes.succeeded:

    for line in result.analyze_result.read_results[0].lines:
        print(line.text)
        print(line.bounding_box)